# L03: NG Feasibility Study

General ssumptions:
- When dispatched, the plant dispatches at full capacity. The dispatch rule is if the revenue would be greater than the cost (when the demand price is greater than the cost?).
- Cost of NG is fixed at \\$2.90/MMBtu
- Efficiency with regards to electricity is .38 fixed for all hours.
- Steam price is \\$15 / MMBtu constant.
- O & M costs are \\$6 / MWh
- Initial capital cost to build the plant is $1000/kW (loan principal). The loan interest is 5% compounded annually, with a period of 20 years.
- For 20 year analysis, inflation rate is 2% per year.
efficiency = 0.38 (electric) fixed for all hours.

In [35]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

In [36]:
# read data
# read into pd dataframe
df = pd.read_csv("./vermont_demand_price_2022.csv")
df2 = pd.read_csv("./Data_wind_temp.csv")

# Extract data
pprice = df["Price ($/MWh)"]
load = df["Load (MW)"]
temp = df2[" temperature (F)"]

In [37]:
# only dispatch steam if rev > cost

def is_dispatched(a,b,c,d):
    if (a + b) > (c + d):
        return 1
    return 0

In [38]:
# steam demand function
# where Hh is the steam demand in hour h, and Th is the outside temperature (in F) in
# hour h. For example, when the temperature is 60 F, the hospital will purchase 1 million
# btu of steam. When the temperature is 0 F, the hospital will purchase up to 7 million
# btu of steam.

hh_list = []
for h in range(8760):
    hh = max((70-temp[h])* .1, 0) # *.1 not 1e5 for UNITS
    hh_list.append(hh)

steam_demand = pd.DataFrame(hh_list, columns=["steam demand per hour"])

In [39]:
# vars
e_ = []
for i in range(8760):
    e_.append(50)

eeout = pd.DataFrame(e_, columns=["eeout"])
s_price = 15 # dollars per MMBtu
install_cost = 1000 # $ per kW (install cost of the thing
ng_price = 2.7 # $/mmbtu
mmbtu_to_mwh = 0.293071
efficiency = .38
o_m = 6 # per MWh

In [40]:
profit_sum = 0
d_factor_list = [] # used to compute hours ON later
gen_hourly = [] # also used later on

elec_rev_sum = 0
steam_rev_sum = 0
fuel_cost_sum = 0
om_cost_sum = 0

for i in range(8760):
    
    ein = eeout.iloc[i,0] / efficiency
    ein_mmbtu = ein / mmbtu_to_mwh

    # revenues
    elec_rev = pprice.iloc[i] * eeout.iloc[i,0]
    max_steam = ein_mmbtu * .4
    steam_rev = s_price * min(steam_demand.iloc[i,0], max_steam)

    # costs
    ng_cost = ng_price * ein / mmbtu_to_mwh
    o_m = 6 * eeout.iloc[i,0] # 6 dollars per MWh

    d_factor = is_dispatched(elec_rev, steam_rev, ng_cost, o_m)
    d_factor_list.append(d_factor)

    gen_hourly.append(eeout.iloc[i,0] * d_factor) # 0 if no dispatch

    profit_sum += (( elec_rev + steam_rev - ng_cost - o_m ) * d_factor)
    elec_rev_sum += elec_rev * d_factor
    steam_rev_sum += steam_rev * d_factor
    fuel_cost_sum += ng_cost * d_factor
    om_cost_sum += o_m * d_factor

print("sum profit (rev-cost): $", profit_sum)
print("sum electricity rev: $", elec_rev_sum)
print("sum steam rev: $", steam_rev_sum)

# should be around 40k or so?

sum profit (rev-cost): $ 23939995.4156468
sum electricity rev: $ 36387255.5
sum steam rev: $ 297633.17064798286


## More Calcualations

In [41]:
# size/cap required to produce average 50 MW/year

# of hours plant produces non-zero energy (dispatch ON hours)
# count hours where dispatch decision is ON and generation > 0

hours_on = sum(d_factor_list)
print("Hours ON:", hours_on)

# capacity factor

cf = hours_on / 8760
print("cf ", cf)

# size/cap required to produce 50MW/yr

cap_MW = 50/cf
print("plant size MW: ", cap_MW)

Hours ON: 8428
cf  0.9621004566210045
plant size MW:  51.969625059326056


In [42]:
# compute annual cost of capital
n = 20
r = .05
P = install_cost * 1e3 * cap_MW

rn = (1+r) ** n

A = P * r * rn / (rn -1)
print("Annual capital pmt: $", A)

Annual capital pmt: $ 4170177.170090506


We model a 20-year outlook using 2% inflation rate.

In [48]:
# inflation rates
inflation = .2

years = range(1, 21) # 20 year outlook
rows = []
for y in years:
    elec_rev = elec_rev_sum * (1 + inflation)**(y-1)
    steam_rev = steam_rev_sum * (1 + inflation)**(y-1)

    fuel_cost = fuel_cost_sum * (1 + inflation) ** (y-1)
    om_cost = om_cost_sum * (1 + inflation) ** (y-1)
    capital_payment = A  # same each year

    total_rev = elec_rev + steam_rev
    total_cost = fuel_cost + om_cost + capital_payment
    net = total_rev - total_cost

    rows.append({
        "Year": y,
        "ElecRev": elec_rev,
        "SteamRev": steam_rev,
        "TotalRev": total_rev,
        "FuelCost": fuel_cost,
        "OMCost": om_cost,
        "CapitalPayment": capital_payment,
        "TotalCost": total_cost,
        "Net": net
    })

df_20yrs = pd.DataFrame(rows)

print(df_20yrs)

    Year       ElecRev      SteamRev      TotalRev      FuelCost  \
0      1  3.638726e+07  2.976332e+05  3.668489e+07  1.021649e+07   
1      2  4.366471e+07  3.571598e+05  4.402187e+07  1.225979e+07   
2      3  5.239765e+07  4.285918e+05  5.282624e+07  1.471175e+07   
3      4  6.287718e+07  5.143101e+05  6.339149e+07  1.765410e+07   
4      5  7.545261e+07  6.171721e+05  7.606979e+07  2.118492e+07   
5      6  9.054314e+07  7.406066e+05  9.128374e+07  2.542190e+07   
6      7  1.086518e+08  8.887279e+05  1.095405e+08  3.050629e+07   
7      8  1.303821e+08  1.066473e+06  1.314486e+08  3.660754e+07   
8      9  1.564585e+08  1.279768e+06  1.577383e+08  4.392905e+07   
9     10  1.877502e+08  1.535722e+06  1.892860e+08  5.271486e+07   
10    11  2.253003e+08  1.842866e+06  2.271432e+08  6.325783e+07   
11    12  2.703604e+08  2.211439e+06  2.725718e+08  7.590940e+07   
12    13  3.244324e+08  2.653727e+06  3.270862e+08  9.109128e+07   
13    14  3.893189e+08  3.184473e+06  3.925034e+

## NG-specific calculations

For the natural gas plant calculate the following:
- The cost of electricity from the plant (USD per MWh electric) when not supplying waste heat.
- The net cost of electricity from the plant (USD per MWh electric) when supplying waste heat to the hospital.
- The number of hours in which the electricity price is higher than the net electricity cost.
- The total revenue from the plant, assuming that the plant only produces when the price is higher than the net cost. This is your dispatch rule.

In [49]:
# electricity cost when no heat

fuel_cost_per_mwh = (1/efficiency) * (1/0.293071) * ng_price
om_cost_per_mwh = om_cost

annual_capital_payment = A
annual_mwh_produced = 50 * 8760
a_capital_per_mwh = annual_capital_payment / annual_mwh_produced

cost_no_heat_per_mwh = fuel_cost_per_mwh + om_cost_per_mwh + a_capital_per_mwh
print("Cost of electricity ($/MWh) when NOT supplying waste heat: ", cost_no_heat_per_mwh)


# net cost when heat
cost_with_heat_per_mwh = (fuel_cost_sum + om_cost_sum + annual_capital_payment - steam_rev_sum) / annual_mwh_produced
print("Cost of electricity ($/MWh) when supplying waste heat: ", cost_with_heat_per_mwh)


# hours profitable

hours_profitable = sum(d_factor_list) # this might be wrong, it hsould be when pprice is higher than net cost

print("hours profitable ", hours_profitable)

Cost of electricity ($/MWh) when NOT supplying waste heat:  80777356.80599074
Cost of electricity ($/MWh) when supplying waste heat:  37.939354462198786
hours profitable  8428
